In [1]:
!pip install flask transformers requests

In [7]:
%%writefile flask_chatbot_app.py
from flask import Flask, request, jsonify
from transformers import pipeline

app = Flask(__name__)

# Load the Hugging Face model

chatbot_model = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)
conversation_history = []

@app.route('/chatbot', methods=['POST'])
def chatbot():
    user_input = request.json.get('message')
    if not user_input:
        return jsonify({'response': 'Please provide a message.'}), 400
    response = generate_response(user_input)
    return jsonify({'response': response})

def generate_response(user_input):
    # Generate a response using the Hugging Face model
    conversation_history.append(
        {"role": "user", "content": user_input},
    )
    result = chatbot_model(conversation_history, num_return_sequences=1, max_new_tokens=250)
    conversation_history.append(
        {"role": "assistant", "content": result[0]['generated_text'][-1]['content']},
    )
    return result[0]['generated_text'][-1]['content']

if __name__ == '__main__':
    app.run(debug=True, port=5000, host='0.0.0.0')


Overwriting flask_chatbot_app.py


In [8]:

import subprocess

# Stop any running Flask app
subprocess.run(['pkill', '-f', 'flask_chatbot_app.py'])

CompletedProcess(args=['pkill', '-f', 'flask_chatbot_app.py'], returncode=1)

In [9]:
!nohup python flask_chatbot_app.py &

OSError: Background processes not supported.

In [10]:
!sudo lsof -i -P -n | grep LISTEN

Password:

In [8]:

import requests

# Define the URL of the Flask app
url = 'http://127.0.0.1:5000/chatbot'

# Send a request to the Flask app
response = requests.post(url, json={'message': 'Hello, how are you?'})
print(response.json())


{'response': " Hello! I'm doing well. How about you? How can I help you today?"}


In [9]:
response = requests.post(url, json={'message': 'I was wondering how can I go to Eiffel Tower from the airport using the train and subway?'})
print(response.json())

{'response': " To travel from Charles de Gaulle Airport (CDG) to the Eiffel Tower using the train and subway, follow these steps:\n\n\n1. **Airport to Gare de l'Est Train Station:**\n\n   - Take the RER B train from Terminal 2 or Terminal 1 to Gare de l'Est. The train ride takes approximately 30 minutes.\n\n   - The train station is located inside Terminal 2, so you'll need to take an airport shuttle bus from Terminal 1 to Terminal 2.\n\n\n2. **Gare de l'Est to Eiffel Tower:**\n\n   - Once you arrive at Gare de l'Est, you'll need to transfer to the Metro.\n\n   - Take Metro Line 1 towards Château de Vincennes and get off at Champ de Mars-Tour Eiffel.\n\n   - The ride on the Metro takes about 15 minutes.\n\n   - The Eiffel Tower is located at Champ de Mars, which is a short walk from the station.\n\n\nPlease note that you'll need to purchase a"}


In [10]:
response = requests.post(url, json={'message': 'And how do I go to Arche du Triomphe from there?'})
print(response.json())

{'response': " To travel from the Eiffel Tower to the Arc de Triomphe, follow these steps:\n\n\n1. **Eiffel Tower to Arc de Triomphe:**\n\n   - From the Eiffel Tower, you have a couple of options to reach the Arc de Triomphe.\n\n   - **Option 1: Walking:** The Arc de Triomphe is a 20-minute walk from the Eiffel Tower. Head west on Avenue de la Bourdonnais, then turn left onto Avenue de la Grande Armée, and continue straight until you reach the Arc de Triomphe.\n\n   - **Option 2: Metro:**\n\n     - Take Metro Line 1 from Champ de Mars-Tour Eiffel to Charles de Gaulle - Étoile.\n\n     - Exit at the Charles de Gaulle - Étoile station.\n\n     - From there, walk west along Avenue de la Grande Armée for about 10 minutes to reach the Arc de Triomphe.\n\n\nPlease note that the Metro option may be more convenient if you prefer not to walk, especially if you're carrying lug"}
